Let's start with importing necessary libraries

In [4]:
### Problem Solution to show arabic letters from right to left
! pip install arabic_reshaper
! pip install python-bidi

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
import json
import tensorflow as tf
import unicodedata
import re
import os

# visualization packages
import matplotlib.pyplot as plt
import seaborn as sns

#sns.color_palette("rocket_r", as_cmap=True)
cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True)
plt.rcParams.update({'font.size': 22})
np.random.seed(42)
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

> Show available files

In [6]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Using `arabic_reshaper` library to reconstruct Arabic sentences to be used in applications that don't support Arabic script.

In [7]:
import arabic_reshaper
from bidi.algorithm import get_display

reshaped_text = arabic_reshaper.reshape(u'لغةٌ عربيّة')
artext = get_display(reshaped_text)

plt.text(0.25, 0.45, artext , name = 'Times New Roman',fontsize=25)

Reading dataset, drop id column that used to btain data texts from [API](https://recruitment.aimtechnologies.co/ai-tasks) then rename columns for our text to tweets. 

In [8]:
df = pd.read_csv("../input/alldialectdataset/out.csv", encoding="utf-8", lineterminator='\n')
df.drop(columns=["id"], inplace=True)
df.columns = ['dialect', 'tweets']
df.tail()

In [9]:
df.describe().T

`dialect` column has 18 unique class and 'tweets' column has some null values to drop and zero dublicated tweets.

In [10]:
df.dropna(inplace=True)

Let's translate` dialect` column in arabic to an `arabic_dialects` column for some visualization plots.

In [11]:
arabic_dialects = {
    'AE': 'لهجة اماراتية',
    'BH': 'لهجة بحرينية',
    'DZ': 'لهجة جزائرية',
    'EG': 'لهجة مصرية',
    'IQ': 'لهجة عراقية',
    'JO': 'لهجة أردنية',
    'KW': 'لهجة كويتية',
    'LB': 'لهجة لبنانية',
    'LY': 'لهجة ليبية',
    'MA': 'لهجة مغربية',
    'OM': 'لهجة عمانية',
    'PL': 'لهجة فلسطينية',
    'QA': 'لهجة قطرية',
    'SA': 'لهجة سعودية',
    'SD': 'لهجة سودانية',
    'SY': 'لهجة سورية',
    'TN': 'لهجة تونسية',
    'YE': 'لهجة يمنية'
}
df['dialect_in_arabic'] = df.dialect.map(arabic_dialects)

Here we create a pie plot to show the percentage of each dialect. This plot shows that the largest percentage for egyptian dialects with 12.6% and the smallest is tunisian dialect with 2.02%.

In [12]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots


values = df.dialect_in_arabic.value_counts();
labels = values.index

fig = go.Figure(
    [
        go.Pie(
            labels = labels,
            values = values
        )
    ]
)
fig.update_layout(title_text="Pie chart of arabic dialects", template="plotly_white")
fig.data[0].marker.colors = [px.colors.qualitative.Plotly[2:]]
fig.data[0].textfont.color = "black"
fig.data[0].textposition = "inside"
fig.show()

Here to visualize top tokens for each dialect we use 'CountVectorizer'.

'CountVectorizer' is a function to convert a collection of text documents to a matrix of token counts.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_tweet_bigrams(df, column_name, dialect, n=None):
    # select all text for selected dialect
    txt = df[df['dialect']==dialect][column_name].str.lower()
    # create a matrix of token counts
    vec = CountVectorizer(ngram_range=(2, 2)).fit(txt)
    # bag of words  
    bag_of_words = vec.transform(txt)
    # sum bag of words 
    sum_words = bag_of_words.sum(axis=0) 
    # frequency of each word in selected texts
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    # top n words repeates in selected texts 
    top = words_freq[:n]
    #print(top)
    x, y = map( list, zip(*top) )
    nn = np.arange(len(x))
    # reshape arabic words
    x = [ get_display( arabic_reshaper.reshape(i)) for i in x]
    
    plt.figure(figsize=(10,8))
    plt.barh(x, y, align='center', alpha=0.2)
    plt.plot(y, nn, '-o', markersize=5, alpha=0.8)
    plt.yticks(nn, x);
    plt.xlabel('Word Number');
    plt.title(f'Top {n} words in {dialect} tweets')

In [14]:
get_top_tweet_bigrams(df, 'tweets', dialect='EG', n=10)
get_top_tweet_bigrams(df, 'tweets', dialect='TN', n=10)
#for dialect in df.dialect.unique():
#    get_top_tweet_bigrams(df, 'tweets', dialect=dialect, n=10)

###### 

### `Clean Text`
Next step to try several methods **one at a time** to clean our tweets like:-

        * Select arabic characters only from text
        * Remove username "@handle" from text
        * Remove URL from text
        * Remove punctuation, emoji and smileys from text
        * Remove \n, \t ,,, etc from text
        * Remove Diacritization from text
        * Remove Arabic Stop Words from text

#### `Select arabic characters only from text`

In [15]:
ar_pattern = r'[\u0600-\u06FF]+'
test_text = df.tweets.iloc[254]
test_text

In [16]:
' '.join(re.findall(ar_pattern, test_text)) # return arabic tokens only

In [17]:
re.sub(ar_pattern, ' ', test_text) # return non arabic tokens

#### `Remove username "@handle"`

In [18]:
handle_pattern = r'(@.*?)[\s]'

In [19]:
re.sub(handle_pattern, ' ', test_text) # return text without @handle

#### `Remove URL`

In [20]:
url_pattern = r'https?://\S+|www\.\S+'
test_text = df.tweets.iloc[1851]
test_text

In [21]:
re.findall(url_pattern, test_text) # return url only

In [22]:
re.compile(r'https?://\S+|www\.\S+').sub(r'', test_text) # return text without url

#### `Remove punctuation, emoji and smileys`

In [23]:
url_pattern = r'[^\w\s]'
test_text = df.tweets.iloc[0]
test_text

In [24]:
re.findall(url_pattern, test_text)

> this do the same as remove emojy. plus it removes punctuation and smily faces

In [25]:
re.sub(url_pattern, '', test_text)

#### `Remove Escape Codes` 
> Escape Codes like \n, \t ,,, etc

In [26]:
url_pattern = r'\s+'
test_text = df.tweets.iloc[12437]
test_text

In [27]:
re.findall(url_pattern,  test_text) # return escape codes found in text

In [28]:
re.sub(url_pattern, ' ', test_text).strip() # return text escape codes found in

#### `Remove Diacritization` (tashkeel)

In [29]:
url_pattern = r'[\u0617-\u061A\u064B-\u0652]'
test_text = df.tweets.iloc[12437]
test_text

In [30]:
re.findall(url_pattern,  test_text)

In [31]:
re.sub(re.compile(url_pattern),"", test_text) # p_tashkeel

#### `Remove Arabic Stop Words`

In [32]:
from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('arabic')
nltk_stopwords[:10]

In [33]:
import urllib.request  # the lib that handles the url stuff
target_url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ar/master/stopwords-ar.txt"

stopwords_ar = []
for line in urllib.request.urlopen(target_url):
    #print(line.decode('utf-8'))
    line = line.decode('utf-8').split("\n")[0]
    stopwords_ar.append(line)
    
stopwords_ar[:10]

In [34]:
test_text = df.tweets.iloc[12437]

def remove_stop_words(text, stopwords):
    words = [word for word in text.split() if word not in stopwords]
    return " ".join(words)    

print(remove_stop_words(test_text, stopwords=stopwords_ar))
print(remove_stop_words(test_text, stopwords=nltk_stopwords))

#### Make a Preprocessing Pipleline from all above methods to try which of them make a positive differnce in score.

In [35]:
def clean_text(text):
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    text = re.compile(r'https?://\S+|www\.\S+').sub(r'', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(re.compile(r'[\u0617-\u061A\u064B-\u0652]'),"", text)
    text = remove_stop_words(text, stopwords_ar)
    text = text.strip()
    return text

df['clean_tweets'] = df.tweets.apply(lambda t: clean_text(t))

Let's visualize top 20 tokens after preprocessing pipline.

In [36]:
get_top_tweet_bigrams(df, 'clean_tweets', dialect='EG', n=20)
get_top_tweet_bigrams(df, 'clean_tweets', dialect='LY', n=20)
#for dialect in df.dialect.unique():
#    get_top_tweet_bigrams(df, 'clean_tweets', dialect=dialect, n=20)

LabelEncoder to encode target labels with value between 0 and n_classes-1 (0 to 17)

In [37]:
#from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['dialect_encode'] = le.fit_transform( df['dialect'] ).astype(np.int8)
target_names = le.inverse_transform(np.arange(18))
target_names

![](https://miro.medium.com/max/1200/1*3-KkZ0hlRZxjMn7Z6uXDGg.png)

# Major regions for dialects
After run several models we notice that Most of misclassifications happens for each dialect with dialects for share border countries. So we can break a classifier into three to five classifiers. each classifier for a group of share border countries.

Here we can Combine dialects with dialects for share border countries. So let's take a small look here.

Starting with define our new regions for dialects

In [47]:
new_regions = {
    'ar_island': ['QA', 'SA', 'AE', 'KW', 'OM', 'YE','BH'],
    'shaam': ['LB', 'SY', 'JO', 'PL', 'IQ'],
    'africa_ne': ['EG', 'LY', 'SD'],
    'africa_nw': ['TN', 'MA', 'DZ'],
}
new_regions_names = new_regions.keys()
# prepare last dictionary for pandas map function to easly store in dataset
new_regions = {val:key for key, lst in new_regions.items() for val in lst}
#new_regions

Create a function to combine deli

In [40]:
new_df = df.copy()
new_df['region'] = new_df['dialect'].map(new_regions)
new_df.head()

In [41]:
new_df.isnull().sum()

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, classification_report, confusion_matrix
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

X = TfidfVectorizer().fit_transform(new_df['tweets'])
y = new_df.region
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.7, shuffle=True, random_state = 42)

model = LinearSVC()
model.fit(X_train, y_train)
predicted = model.predict(X_val)
precision_score(predicted, y_val, average='micro')

In [50]:
def ConfusionMatrix(cm, target_names):
    #cm = confusion_matrix(y_val, predicted)
    df_cm = pd.DataFrame(cm, target_names, target_names)

    plt.figure(figsize = (10, 10))
    sns.set(font_scale=1.4) # for label size
    sns.heatmap(
        df_cm,
        fmt=".1f", annot=True, annot_kws={'size': 12}, cmap=cmap,
        linewidths=3,
    );
    return None

ConfusionMatrix(confusion_matrix(y_val, predicted), new_regions_names)

> We can try to reallocate dialects in diffrent regions for better score.

> ## `Furthermore we can break a classifier into three to five classifiers to classiffy dialects in the same regions.`

![](https://miro.medium.com/max/1200/1*3-KkZ0hlRZxjMn7Z6uXDGg.png)

In [ ]:
print("let's stop here for now")
break;

### t-SNE Corpus Visualization
        One very popular method for visualizing document similarity is to use t-distributed stochastic neighbor embedding. By decomposing high-dimensional document vectors into 2 dimensions using probability distributions from both the original dimensionality and the decomposed dimensionality, t-SNE is able to effectively cluster similar documents. By decomposing to 2 or 3 dimensions, the documents can be visualized with a scatter plot.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
from yellowbrick.text import TSNEVisualizer

In [ ]:
Vectorized_data = TfidfVectorizer().fit_transform(df['clean_tweets'])
y = df.dialect

# Create the visualizer and draw the vectors
tsne = TSNEVisualizer()
tsne.fit(Vectorized_data, y)
tsne.show()

Note that you can pass the class labels or document categories directly to the TSNEVisualizer as follows:

In [ ]:
tsne = TSNEVisualizer(labels=labels)
tsne.fit(X, y)
tsne.show()